### Exercise: `read_version_payload`

In [ ]:
def read_version_payload(stream):
    # We will build up this dictionary as we go
    r = {}
    
    # First read the 4 byte `version` number and save to the r['version'] key 
    r['version'] = stream.read(4)
    
    # Your turn: follow this pattern to fill in the "timestamp", "receiver_address", "sender_address", and "nonce" fields
    r['services'] = stream.read(8)
    r['timestamp'] = stream.read(8)
    r['receiver_address'] = stream.read(26)
    r['sender_address'] = stream.read(26)
    r['nonce'] = stream.read(8)
    
    # I will do the "user_agent" attribute for you. You will re-implement later ...
    r['user_agent'] = magic_read_varstr(stream)
    
    # Your turn: Fill out the remaining "start_height" and "relay" attributes
    r['start_height'] = stream.read(4)
    r['relay'] = stream.read(1)
        
    # Return the dictionary we've assembled
    return r

### Exercise: `little_endian_to_int(bytes)` and `big_endian_to_int(bytes)` 


In [ ]:
def little_endian_to_int(b):
    return int.from_bytes(b, 'little')

def big_endian_to_int(b):
    return int.from_bytes(b, 'big')

### Exercise: Update `read_version_payload` to interpret integers using `little_endian_to_int` and `big_endian_to_int`

In [ ]:
def read_version_payload(stream):
    r = {}    
    r['version'] = little_endian_to_int(stream.read(4))
    r['services'] = little_endian_to_int(stream.read(8))
    r['timestamp'] = little_endian_to_int(stream.read(8))
    r['receiver_address'] = stream.read(26)
    r['sender_address'] = stream.read(26)
    r['nonce'] = little_endian_to_int(stream.read(8))
    r['user_agent'] = magic_read_varstr(stream)
    r['start_height'] = little_endian_to_int(stream.read(4))
    r['relay'] = stream.read(1)
    return r

### Exercise: Given a version message payload (a dictionary of the sort returned by `read_version_payload`), determine whether it's node can send a `pong` message


In [ ]:
def can_send_pong(version_payload):
    return version_payload['version'] >= 60001

### Exercise: Given a version message payload (dictionary of the sort returned by `read_version_payload`), tell me if it is from the last hour or not

In [ ]:
def is_less_than_one_hour_old(version_payload_dict):
    """Using time.time()"""
    return version_payload_dict['timestamp'] > time.time() - 60*60

def is_less_than_one_hour_old(version_payload_dict):
    """Using datetime"""
    dt = datetime.fromtimestamp(version_payload_dict['timestamp'])
    one_hour_ago = datetime.now() - timedelta(hours=1)
    return dt > one_hour_ago

### Exercise: `check_bit(bitfield, index)`

In [ ]:
def check_bit(bitfield, index):
    mask = 1 << index
    return bool(bitfield & mask)

### Exercise:  `services_int_to_dict`

In [ ]:
def services_int_to_dict(services_int):
    return {
        'NODE_NETWORK': check_bit(services_int, 0),
        'NODE_GETUTXO': check_bit(services_int, 1),
        'NODE_BLOOM': check_bit(services_int, 2),
        'NODE_WITNESS': check_bit(services_int, 3),
        'NODE_NETWORK_LIMITED': check_bit(services_int, 10),
    }


### Exercise: Complete these function definitions to hammer home you understanding of this strange `services` "bitfield"

In [ ]:
def offers_node_network_service(services_bitfield):
    services_dict = services_int_to_dict(services_bitfield)
    return services_dict['NODE_NETWORK']

In [ ]:
def offers_node_bloom_and_node_witness_services(services_bitfield):
    services_dict = services_int_to_dict(services_bitfield)
    return services_dict['NODE_BLOOM'] and services_dict['NODE_WITNESS']

### Exercise: `bytes_to_bool(bytes)`

In [ ]:
def bytes_to_bool(bytes):
    return bool(little_endian_to_int(bytes))

### Exercise: Use `bytes_to_bool` to the interpret the `relay` bytes in `read_version_payload`

In [ ]:
def read_version_payload(stream):
    r = {}    
    r['version'] = little_endian_to_int(stream.read(4))
    r['services'] = little_endian_to_int(stream.read(8))
    r['timestamp'] = little_endian_to_int(stream.read(8))
    r['receiver_address'] = stream.read(26)
    r['sender_address'] = stream.read(26)
    r['nonce'] = little_endian_to_int(stream.read(8))
    r['user_agent'] = magic_read_varstr(stream)
    r['start_height'] = little_endian_to_int(stream.read(4))
    r['relay'] = bytes_to_bool(stream.read(1))
    return r

### Exercise:  Implement `read_varint`, since `read_varstr` will depend on it and the version message's `user_agent` requires `read_varstr`

In [ ]:
def read_varint(stream):
    i = little_endian_to_int(stream.read(1))
    if i == 0xff:
        return little_endian_to_int(stream.read(8))
    elif i == 0xfe:
        return little_endian_to_int(stream.read(4))
    elif i == 0xfd:
        return little_endian_to_int(stream.read(2))
    else:
        return i

### Exercise: Implement `read_varstr`


In [ ]:
def read_varstr(stream):
    length = read_varint(stream)
    string = stream.read(length)
    return string

### Exercise: `read_version_payload` calls your `read_varstr`


In [ ]:
def read_version_payload(stream):
    r = {}    
    r['version'] = little_endian_to_int(stream.read(4))
    r['services'] = little_endian_to_int(stream.read(8))
    r['timestamp'] = little_endian_to_int(stream.read(8))
    r['receiver_address'] = stream.read(26)
    r['sender_address'] = stream.read(26)
    r['nonce'] = little_endian_to_int(stream.read(8))
    r['user_agent'] = read_varstr(stream)
    r['start_height'] = little_endian_to_int(stream.read(4))
    r['relay'] = bytes_to_bool(stream.read(1))
    return r

### Exercise: Read an IPv4 address

In [28]:
# Reading an IPv4 address
ipv4_bytes = b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\n\x00\x00\x01'

In [29]:
# First, throw away the prefix
ip = ipv4_bytes[12:]
ip

b'\n\x00\x00\x01'

In [30]:
# Interpret each byte as an integer
ip = list(ip)
ip

[10, 0, 0, 1]

In [32]:
# String-formatting

'.'.join([str(char) for char in ip])

'10.0.0.1'

### Exercise: Read an IPv6 Address

In [24]:
ipv6_bytes = b'\xfe\x80\x00\x00\x00\x00\x00\x00\x02\x02\xb3\xff\xfe\x1e\x83)'
ipv6_bytes

b'\xfe\x80\x00\x00\x00\x00\x00\x00\x02\x02\xb3\xff\xfe\x1e\x83)'

In [25]:
# Grab pairwise bytes

ip = [ipv6_bytes[i:i+2] for i in range(0, 16, 2)]
ip

[b'\xfe\x80',
 b'\x00\x00',
 b'\x00\x00',
 b'\x00\x00',
 b'\x02\x02',
 b'\xb3\xff',
 b'\xfe\x1e',
 b'\x83)']

In [26]:
# Turn each byte-pair into hex strings

ip = [two_bytes.hex() for two_bytes in ip]
ip

['fe80', '0000', '0000', '0000', '0202', 'b3ff', 'fe1e', '8329']

In [27]:
# String formatting

ip = ':'.join(ip)
ip

'fe80:0000:0000:0000:0202:b3ff:fe1e:8329'

### Exercise: `bytes_to_ip`

In [ ]:
def bytes_to_ip(b):
    # IPv4
    if b[0:12] == IPV4_PREFIX:
        return socket.inet_ntop(socket.AF_INET, b[12:16])
    # IPv6
    else:
        return socket.inet_ntop(socket.AF_INET6, b)

### Exercise: `read_address`


In [ ]:
def read_address(stream, has_timestamp):
    r = {}
    if has_timestamp:
        r["timestamp"] = little_endian_to_int(stream.read(4))
    r["services"] = little_endian_to_int(stream.read(8))
    r["ip"] = bytes_to_ip(stream.read(16))
    r["port"] = big_endian_to_int(stream.read(2))
    return r

### Exercise: final `read_version_payload`

In [ ]:
def read_version_payload(stream):
    r = {}
    r["version"] = little_endian_to_int(stream.read(4))
    r["services"] = little_endian_to_int(stream.read(8))
    r["timestamp"] = little_endian_to_int(stream.read(8))
    r["receiver_address"] = read_address(stream, has_timestamp=False)
    r["sender_address"] = read_address(stream, has_timestamp=False)
    r["nonce"] = little_endian_to_int(stream.read(8))
    r["user_agent"] = read_varstr(stream)
    r["start_height"] = little_endian_to_int(stream.read(4))
    r["relay"] = little_endian_to_int(stream.read(1))
    return r